# Best Estimate

The best estimate I can do with current information, making use of meter readings collected over the past year, as well as some climate data on hours of sunshine from the Met Office, and some basic assumptions about costs, unit rates and solar panel outputs.

In [2]:
import json
import pandas as pd
import yaml

In [3]:
with open("../data/ballpark_figures.yaml", "r", encoding="utf-8") as filepath:
    parameters = yaml.load(filepath, Loader=yaml.FullLoader)

ballpark_installation_cost = parameters["installation_cost"]
ballpark_sell_back_rate = parameters["sell_back_rate"]
expected_unit_rate = parameters["expected_unit_rate"]
solar_panel_total_output = parameters["solar_panel_total_output"]
solar_panel_lifetime = parameters["solar_panel_lifetime"]

print(json.dumps(parameters, indent=4))

{
    "installation_cost": 30000,
    "expected_unit_rate": 0.2839,
    "sell_back_rate": 0.1766,
    "yearly_bill": 14126.03,
    "ave_hours_daylight": 3.44,
    "solar_panel_total_output": 4.4,
    "solar_panel_lifetime": 25
}


In [4]:
# Meter readings collected from May 2023 - May 2024:
meter_readings_df = pd.read_excel(
    "~/Documents/committee/meter_readings.xlsx", sheet_name="meter_readings", usecols=[0, 1, 2]
)

# Data on hours of sunshine gathered from Met Office (https://www.metoffice.gov.uk/research/climate/maps-and-data/uk-climate-averages/gcey2u2yw)
hours_of_sunshine_df = pd.read_excel("~/Documents/committee/meter_readings.xlsx", sheet_name="hours_of_sunshine")

In [5]:
meter_readings_df.head()

,date,meter_reading,weekly_usage
0,2023-04-30,59059.0,NaN
1,2023-05-07,59127.0,68.0
2,2023-05-14,59225.0,98.0
3,2023-05-21,59353.0,128.0
4,2023-05-28,59467.0,114.0


In [6]:
hours_of_sunshine_df

,month,hours_of_sunshine,daily_average
0,January,42.70,1.377419
1,February,66.93,2.369204
2,March,101.15,3.262903
3,April,148.19,4.939667
4,May,183.30,5.912903
5,June,150.13,5.004333
6,July,136.14,4.391613
7,August,136.15,4.391935
8,September,112.91,3.763667
9,October,85.41,2.755161


In [7]:
meter_readings_df["month"] = meter_readings_df["date"].apply(lambda x: x.month_name())

monthly_usage_df = meter_readings_df.groupby("month", as_index=False)["weekly_usage"].sum()
monthly_usage_df.rename({"weekly_usage": "monthly_usage"}, axis=1, inplace=True)
monthly_usage_df["average_daily_usage"] = monthly_usage_df["monthly_usage"] / [31, 28.25, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

In [9]:
# Multiplying solar panel output by hours of sunshine gives output for month:
hours_of_sunshine_df["solar_panel_output"] = hours_of_sunshine_df.hours_of_sunshine.apply(lambda x: x * solar_panel_total_output)

# Divide by number of days to get daily average:
hours_of_sunshine_df["average_daily_output"] = hours_of_sunshine_df["solar_panel_output"] / [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

hours_of_sunshine_df

,month,hours_of_sunshine,daily_average,solar_panel_output,average_daily_output
0,January,42.70,1.377419,187.880,6.060645
1,February,66.93,2.369204,294.492,10.517571
2,March,101.15,3.262903,445.060,14.356774
3,April,148.19,4.939667,652.036,21.734533
4,May,183.30,5.912903,806.520,26.016774
5,June,150.13,5.004333,660.572,22.019067
6,July,136.14,4.391613,599.016,19.323097
7,August,136.15,4.391935,599.060,19.324516
8,September,112.91,3.763667,496.804,16.560133
9,October,85.41,2.755161,375.804,12.122710


In [10]:
# By merging datasets on month, we can see how usage compares with generation:
output_and_usage = pd.merge(hours_of_sunshine_df, monthly_usage_df, how="inner", on="month")

,month,hours_of_sunshine,daily_average,solar_panel_output,average_daily_output,monthly_usage,average_daily_usage
0,January,42.70,1.377419,187.880,6.060645,743.0,23.967742
1,February,66.93,2.369204,294.492,10.517571,782.0,26.066667
2,March,101.15,3.262903,445.060,14.356774,945.0,30.483871
3,April,148.19,4.939667,652.036,21.734533,686.0,22.129032
4,May,183.30,5.912903,806.520,26.016774,580.0,19.333333
5,June,150.13,5.004333,660.572,22.019067,379.0,12.225806
6,July,136.14,4.391613,599.016,19.323097,450.0,15.000000
7,August,136.15,4.391935,599.060,19.324516,483.0,17.097345
8,September,112.91,3.763667,496.804,16.560133,564.0,18.193548
9,October,85.41,2.755161,375.804,12.122710,917.0,30.566667


We can see the figures for usage and generation by month below. As a side-note, the usage (shown in `average_daily_usage`), particularly during the summer months, seems very high to me, so I'd be curious to know what others think.

In [11]:
output_and_usage

,month,hours_of_sunshine,daily_average,solar_panel_output,average_daily_output,monthly_usage,average_daily_usage
0,January,42.70,1.377419,187.880,6.060645,743.0,23.967742
1,February,66.93,2.369204,294.492,10.517571,782.0,26.066667
2,March,101.15,3.262903,445.060,14.356774,945.0,30.483871
3,April,148.19,4.939667,652.036,21.734533,686.0,22.129032
4,May,183.30,5.912903,806.520,26.016774,580.0,19.333333
5,June,150.13,5.004333,660.572,22.019067,379.0,12.225806
6,July,136.14,4.391613,599.016,19.323097,450.0,15.000000
7,August,136.15,4.391935,599.060,19.324516,483.0,17.097345
8,September,112.91,3.763667,496.804,16.560133,564.0,18.193548
9,October,85.41,2.755161,375.804,12.122710,917.0,30.566667


Let's assume now that if we install a 10kWh battery, then the first 10kWh generated will be stored in the battery to be used, and any electricity generated above this will be sold to the grid. 

This is an approximation, and may not be totally accurate; it's an attempt to account for the predominant use in the evenings so that if we generate 10kWh by 3pm for example, no more electricity generated after that can be kept until the battery starts to be depleted. During the summer time I imagine most use is during Sunday morning, so this would not hold then, but during the week if the halls were not in use, then it would be valid.

In [87]:
output_and_usage["average_daily_units_to_sell"] = output_and_usage["average_daily_output"].apply(lambda x: max(0, x - 10))
output_and_usage["average_daily_units_to_use"] = output_and_usage["average_daily_output"].apply(lambda x: min(x, 10))

# Converting back to monthly:
output_and_usage["average_monthly_units_to_sell"] = output_and_usage["average_daily_units_to_sell"] * [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
output_and_usage["average_monthly_units_to_use"] = output_and_usage["average_daily_units_to_use"] * [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

output_and_usage

,month,hours_of_sunshine,daily_average,solar_panel_output,average_daily_output,monthly_usage,average_daily_usage,average_daily_units_to_sell,average_daily_units_to_use,average_monthly_units_to_sell,average_monthly_units_to_use
0,January,42.70,1.377419,187.880,6.060645,743.0,23.967742,0.000000,6.060645,0.000,187.880
1,February,66.93,2.369204,294.492,10.517571,782.0,26.066667,0.517571,10.000000,14.492,280.000
2,March,101.15,3.262903,445.060,14.356774,945.0,30.483871,4.356774,10.000000,135.060,310.000
3,April,148.19,4.939667,652.036,21.734533,686.0,22.129032,11.734533,10.000000,352.036,300.000
4,May,183.30,5.912903,806.520,26.016774,580.0,19.333333,16.016774,10.000000,496.520,310.000
5,June,150.13,5.004333,660.572,22.019067,379.0,12.225806,12.019067,10.000000,360.572,300.000
6,July,136.14,4.391613,599.016,19.323097,450.0,15.000000,9.323097,10.000000,289.016,310.000
7,August,136.15,4.391935,599.060,19.324516,483.0,17.097345,9.324516,10.000000,289.060,310.000
8,September,112.91,3.763667,496.804,16.560133,564.0,18.193548,6.560133,10.000000,196.804,300.000
9,October,85.41,2.755161,375.804,12.122710,917.0,30.566667,2.122710,10.000000,65.804,310.000


In [91]:
def get_expected_saving(
        monthly_units_to_sell,
        monthly_units_to_use,
        monthly_usage,
        unit_rate=expected_unit_rate,
        sell_back_rate=ballpark_sell_back_rate
    ):
    """
    Return expected saving given stats for monthly output, monthly usage,
    expected unit rate and sell back rate.

    We take account of battery capacity so that the savings are only based on the units we would
    have available to use, plus any money gained from selling leftover units back to the grid.
    """

    # doesn't appear to be the case, but if we were to use less than what is stored in the battery,
    # we'd sell that as well:
    if monthly_usage < monthly_units_to_use:
        monthly_units_to_sell += (monthly_units_to_use - monthly_usage)
        return (monthly_usage * unit_rate) + (monthly_units_to_sell * sell_back_rate)
    return (monthly_units_to_use * unit_rate) + (monthly_units_to_sell * sell_back_rate)

output_and_usage[f"expected_saving_{expected_unit_rate}"] = output_and_usage.apply(
    lambda x: get_expected_saving(
        x["average_monthly_units_to_sell"],
        x["average_monthly_units_to_use"],
        x["monthly_usage"],
    ),
    axis=1
)

In [92]:
output_and_usage

,month,hours_of_sunshine,daily_average,solar_panel_output,average_daily_output,monthly_usage,average_daily_usage,average_daily_units_to_sell,average_daily_units_to_use,average_monthly_units_to_sell,average_monthly_units_to_use,expected_saving_0.2839
0,January,42.70,1.377419,187.880,6.060645,743.0,23.967742,0.000000,6.060645,0.000,187.880,53.339132
1,February,66.93,2.369204,294.492,10.517571,782.0,26.066667,0.517571,10.000000,14.492,280.000,82.100560
2,March,101.15,3.262903,445.060,14.356774,945.0,30.483871,4.356774,10.000000,135.060,310.000,112.319800
3,April,148.19,4.939667,652.036,21.734533,686.0,22.129032,11.734533,10.000000,352.036,300.000,148.536480
4,May,183.30,5.912903,806.520,26.016774,580.0,19.333333,16.016774,10.000000,496.520,310.000,177.382600
5,June,150.13,5.004333,660.572,22.019067,379.0,12.225806,12.019067,10.000000,360.572,300.000,150.072960
6,July,136.14,4.391613,599.016,19.323097,450.0,15.000000,9.323097,10.000000,289.016,310.000,140.031880
7,August,136.15,4.391935,599.060,19.324516,483.0,17.097345,9.324516,10.000000,289.060,310.000,140.039800
8,September,112.91,3.763667,496.804,16.560133,564.0,18.193548,6.560133,10.000000,196.804,300.000,120.594720
9,October,85.41,2.755161,375.804,12.122710,917.0,30.566667,2.122710,10.000000,65.804,310.000,99.853720


If we assume a unit rate of £0.2839, and an installation cost of ~ £30,000 (accounting for ~ £10,000 for panel installation, and two payments of £10,000 each for an initial 10kWh battery and a replacement after ~10-15 years), then we can calculate a rough estimate of annual savings and a breakeven point:

In [101]:
estimated_annual_saving = round(output_and_usage['expected_saving_0.2839'].sum(), 2)

print(f"Estimated annual saving assuming a unit rate of £{expected_unit_rate}: £{estimated_annual_saving}")
print(
    "Estimated time until breakeven assuming total installation cost of £30,000: "
    f"{round(ballpark_installation_cost / estimated_annual_saving, 2)} years."
)

Estimated annual saving assuming a unit rate of £0.2839: £1339.88
Estimated time until breakeven assuming total installation cost of £30,000: 22.39 years.


#### Other things to note:

- Solar panel maintenance: we may need to spend £100-200 per year to clean and service panels for example.
- Efficiency degradation: according to [this link](https://www.ecowatch.com/solar/solar-panel-efficiency-over-time#:~:text=The%20average%20degradation%20is%202.5,and%2085.5%25%20after%2025%20years.), efficiency decreases by 0.5% per year. If for now we assume that annual savings are directly proportional to panel output, (a rough approximation), then we can update our calculations (see below). This shouldn't make a big difference to the final conclusion.

In [113]:
savings_by_year = []
outstanding_cost_by_year = []

outstanding_cost = ballpark_installation_cost
for year in range(1, solar_panel_lifetime + 1):
    savings = estimated_annual_saving * 0.995**year
    savings_by_year.append(estimated_annual_saving * 0.995**year)
    outstanding_cost -= savings
    outstanding_cost_by_year.append(outstanding_cost)

pd.DataFrame(
    {
        "year": list(range(1, solar_panel_lifetime + 1)),
        "annual_savings": savings_by_year,
        "outstanding_cost": outstanding_cost_by_year
    }
)


,year,annual_savings,outstanding_cost
0,1,1333.180600,28666.819400
1,2,1326.514697,27340.304703
2,3,1319.882124,26020.422579
3,4,1313.282713,24707.139867
4,5,1306.716299,23400.423567
5,6,1300.182718,22100.240849
6,7,1293.681804,20806.559045
7,8,1287.213395,19519.345650
8,9,1280.777328,18238.568322
9,10,1274.373442,16964.194880
